# BAI-Assistent

Grundidee: Hilft bei der Erstellung von Zusammenfassungen, welche auf Basis unseren Zusammenfassungen und Vorlesungsfolien die Antworten generiert

#### Unsere Problemstellung
Während der Prüfungsvorbereitung sind vor allem Erstsemester Studenten überfordert, wie man beim Lernen vorgehen kann. Daher haben wir es als Lücke vor allem im BAI-Studiengang erkannt.

#### Use Cases: 
Ich möchte, dass mir der Lernassistent mir Fachbegriffe in ML und Einführung KI erklärt
Ich möchte gut auf die Prüfungen durch den Lernassistenten vorbereitet werden
-	Ich möchte Prüfungsfragen erhalten
-	Ich möchte, dass es Merksätze gibt
-	Ich möchte, dass die Erklärungen einfach sind
-	Ich möchte Hilfe/Beratung erhalten, wie ich mein Cheat Sheet gemäss Stoffabgrenzung aufstellen kann
Ich möchte schnelle und unlimitierte Antworten
(Ich möchte Prüfungsfragen vom Chatbot erhalten, damit ich mich gut auf die Prüfung vorbereiten kann)

##### Zielgruppe: 
BAI-Studenten im ersten Studienjahr, die Maschinelles Lernen und Einführung in die Künstliche Intelligenz belegen

##### KPIs: 
•	Antwortzeit < 5 Sekunden
•	Prüfungsnutzen > 70 % finden Quiz hilfreich
•	Fachliche Korrektheit >85%

Unsere Erwartungen: Fachbegriffe fragen, Unterschied zwischen Supervised und Unsupervised Learning, Was ist One Hot Encoding
Inhalte für den KI-Assistenten: Folien Unterricht, Zusammenfassungen, Stoffabgrenzung


#### Zusammenführung LLM und API

In [1]:
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

# --- Stelle sicher, dass dieser Key existiert ---
assert "GROQ_API_KEY_BAI" in os.environ, "GROQ_API_KEY fehlt in den Env Vars!"

# --- Initialisiere LLM explizit für OpenRouter ---
llm = ChatOpenAI(
    model="openai/gpt-oss-120b",   
    api_key=os.environ["GROQ_API_KEY_BAI"],
    base_url="https://api.groq.com/openai/v1",
    temperature=0.3,
)

print("Sende Test-Ping...")
try:
    msg = llm.invoke("Sag exakt: pong")
    print("Antworttyp:", type(msg))
    # msg ist i.d.R. ein AIMessage – gib Inhalt sicher aus:
    print("Inhalt:", getattr(msg, "content", msg))
except Exception as e:
    print("FEHLER beim LLM-Aufruf:", repr(e))


C:\Users\amuel\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Sende Test-Ping...
Antworttyp: <class 'langchain_core.messages.ai.AIMessage'>
Inhalt: pong


In [2]:
import os
import langchain
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI


#LLM_MODEL = "openai/gpt-oss-20b:free"
LLM_MODEL = "openai/gpt-oss-120b"
LLM_TEMPERATURE = 0.4
BASE_URL = "https://api.groq.com/openai/v1"
OPENROUTER_API_KEY = os.getenv("GROQ_API_KEY_BAI")
USER_PROMPT="Ich verstehe GenAI nicht, kannst du das mir einfach erklären?"

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate


llm = ChatOpenAI(
    model=LLM_MODEL,
    temperature=LLM_TEMPERATURE,
    base_url=BASE_URL,
    api_key=OPENROUTER_API_KEY,
)

print(type(llm))


<class 'langchain_openai.chat_models.base.ChatOpenAI'>


#### Kurz sicherstellen, ob API Key funktioniert

In [4]:
try:
    print(llm.invoke("Sag nur: pong").content)
except Exception as e:
    print(repr(e))

# Test 2: Env-Variablen sichtbar?
import os
print("OPENAI_API_KEY" in os.environ, os.environ.get("OPENAI_BASE_URL"))
print("GROQ_API_KEY_BAI" in os.environ)
print("OPENROUTER_API_KEY" in os.environ)

pong
False None
True
False


#### ChatPrompt Template

In [5]:
from langchain_core.prompts import ChatPromptTemplate
 
LERNASSISTENT_PROMPT = ChatPromptTemplate.from_messages([
    (
        "system",
        (
            "Sprache: Deutsch. Rolle: FHNW-BAI-Lernassistent; erkläre wie eine geduldige Lehrperson.\n"
            "Nutze AUSSCHLIESSLICH den bereitgestellten CONTEXT (Folien/Skripte).\n"
            "Wenn Informationen fehlen oder die Frage nicht im CONTEXT abgedeckt ist, antworte exakt:\n"
            "\"Dazu habe ich im bereitgestellten Material nichts.\" \n"
            "Schlage danach präzise nächste Schritte vor (z. B. welche Folie/Abschnitt hochzuladen wäre).\n"
            "Ziel: Studierende effizient auf Prüfungen vorbereiten.\n"
            "Stil: aktiv, konkret, ohne Floskeln, keine Gender-Sonderzeichen (nutze z. B. 'Lehrperson').\n"
            "Gib GENAU EINEN Lösungsvorschlag und EIN einfaches Beispiel.\n"
            "Halte dich an Terminologie aus dem CONTEXT. Keine externen Fakten, keine Spekulation.\n"
            "CONTEXT:\n{context}"
        )
    ),
    (
        "human",
        (
            "FRAGE: {question}\n"
            "Erstelle die Antwort in genau dieser Struktur:\n"
            "1) Kurzantwort (2–3 Sätze, prüfungsrelevant)\n"
            "2) Erklärung (max. 8 Sätze, schrittweise, mit Intuition)\n"
            "3) Beispiel (sehr einfach, kleine Zahlen/konkreter Mini-Fall)\n"
            "4) Typische Prüfungsfehler (Bullets)\n"
            "5) Verständnis-Check (1–2 Kontrollfragen)\n"
            "6) Quellen (Dokumenttitel + Seiten/Abschnitt aus CONTEXT)"
        )
    ),
])
 

#### PDF Loader

In [6]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyMuPDFLoader  
from pathlib import Path
 
pdf_dir = Path("data/pdfs") #zeigt wo die PDFs gespeichert sind
 
pdf_files = [
    "01 KI Ueberblick Teil 1.pdf",
    "01 KI Ueberblick Teil 2.pdf",
    "02_Problemloesen_als_Suche.pdf",
    "03 Machine Learning_exam.pdf",
    "03 Machine Learning.pdf",
    "03-1 Wissensrepraesentation.pdf",
    "03-2_Aussagenlogik.pdf",
    "03-3_Praedikatenlogik.pdf",
    "05 Deep Learning_exam.pdf",
    "05 Deep Learning.pdf",
    "06 GenAI LLMs.pdf"
]
  
# all_pages_pdf = []
# for name in pdf_files:
#     pdf_path = pdf_dir / name  
#     if not pdf_path.exists():
#         print(f"Datei nicht gefunden: {pdf_path}")
#         continue
#     loader = PyPDFLoader(str(pdf_path))
#     pages = loader.load()
#     all_pages_pdf.extend(pages)
 
# print(f"Loaded {len(all_pages_pdf)} pages from {len(pdf_files)} PDF documents.")
 

all_pages_pdf = []

for name in pdf_files:
    pdf_path = pdf_dir / name
    if not pdf_path.exists():
        print(f"❌ Datei nicht gefunden: {pdf_path}")
        continue

    loader = PyMuPDFLoader(str(pdf_path))
    pages = loader.load()
    all_pages_pdf.extend(pages)
    print(f"✅ {name}: {len(pages)} Seiten geladen")

print(f"\n📚 Insgesamt {len(all_pages_pdf)} Seiten aus {len(pdf_files)} PDF-Dateien geladen.")



✅ 01 KI Ueberblick Teil 1.pdf: 33 Seiten geladen
✅ 01 KI Ueberblick Teil 2.pdf: 72 Seiten geladen
✅ 02_Problemloesen_als_Suche.pdf: 35 Seiten geladen
✅ 03 Machine Learning_exam.pdf: 48 Seiten geladen
✅ 03 Machine Learning.pdf: 78 Seiten geladen
✅ 03-1 Wissensrepraesentation.pdf: 11 Seiten geladen
✅ 03-2_Aussagenlogik.pdf: 53 Seiten geladen
✅ 03-3_Praedikatenlogik.pdf: 26 Seiten geladen
✅ 05 Deep Learning_exam.pdf: 33 Seiten geladen
✅ 05 Deep Learning.pdf: 39 Seiten geladen
✅ 06 GenAI LLMs.pdf: 46 Seiten geladen

📚 Insgesamt 474 Seiten aus 11 PDF-Dateien geladen.


In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# get both websites and pdfs together
all_docs = pages

# define the splitter and strategy
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=100)
splits = splitter.split_documents(all_docs)

In [8]:
import numpy as np

lengths = [len(s.page_content) for s in splits]
print(f"Initial documents: {len(all_docs)}")
print(f"Total chunks: {len(splits)}")
print(f"Avg length: {np.mean(lengths):.1f}")
print(f"Min: {np.min(lengths)}, Max: {np.max(lengths)}")

Initial documents: 46
Total chunks: 111
Avg length: 225.4
Min: 47, Max: 300


In [9]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


C:\Users\amuel\AppData\Local\Temp\ipykernel_26236\4293025327.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


In [10]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

embedding_dim = len(embeddings.embed_query("hello world"))
index = faiss.IndexFlatL2(embedding_dim)

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
    normalize_L2=True
)

vector_store.add_documents(documents=splits)

['e48ec05f-f220-499e-9a5c-2e37c083efd2',
 'ffe6e2ec-2238-4594-b831-cd7b3b53dec7',
 'a44770a8-547a-47c4-b251-7a86fbe851f6',
 'e557b4b0-ded6-44e3-bcdb-0a4ea755071b',
 '0238679d-cbaf-4850-ae76-650be7b2b9f7',
 '43f365d3-9024-4a59-91b0-6dad31ae7c6d',
 'eca20cf3-efd3-4318-8a2d-10e35754df43',
 '49a74535-31c5-4da4-aabe-9948f1303d5f',
 'db0e2845-add3-4034-ae7c-24f76f136a51',
 'f176e968-69bb-41be-b819-e5e7f5997f1e',
 '770cdf94-f441-49ac-8552-fa80696a5a11',
 'a377399e-62e8-467e-8b73-d0e30e535eec',
 'b8ced790-34af-4b04-84b7-eb06cca4e161',
 '3def7681-c420-45ff-9772-ede09c45476b',
 '8bd90d15-a1e1-4f2a-a37a-ee64cfc0ef3a',
 '83114681-1fed-4f75-a454-a36b005d57be',
 '6995032c-04f4-4e70-b2a8-3641824e4cc5',
 'd6955563-fb13-421f-aad8-1f23b15ae385',
 '0481717c-b97b-448c-8dbd-ca54a2b6ca26',
 'cf005615-ef59-4c06-bc4c-d2e71b4433c1',
 '5ce36508-53f3-4eef-905a-37cf2c8e4a7e',
 'a5a5b5ca-ba6f-4c4b-9f8e-4801627a94f4',
 'e5755230-1764-4c77-bf17-92a62903e9f0',
 '78b07d1f-f75f-4a76-b7ee-4236c92c4ade',
 '5b50a0ee-c65e-

In [11]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [12]:
docs = retriever.get_relevant_documents("Prädikatenlogik")
for i, d in enumerate(docs, 1):
    print(f"--- Retrieved doc {i} ---")
    print(d.page_content[:400], "...\n")

--- Retrieved doc 1 ---
w  d n.
– I  B   ich d   Bildv  a   itung w  d n s lch  
M d ll   ü  Au ga  n wi  Bildklassi izi  ung,
O j kt  k nnung  d   Bildg n  i  ung  ing s tzt.
– In d   Audi v  a   itung  ind n sich 
Anw ndung n in Sp ach  k nnung, 
Musikk  p siti n  d   S undanalys . ...

--- Retrieved doc 2 ---
Prof. Dr. Manuel Renold
Was ist in LLMs drin?
GenAI & LLMs
28
Bild: https://lifearchitect.ai/whats-in-my-ai/ ...

--- Retrieved doc 3 ---
Prof. Dr. Manuel Renold
– T  p  atu : R guli  t di  Zu älligk it d   Antw  t n  in s Sp ach  d lls.
– Ein luss: Höh    W  t    höh n di  Vi l alt, ni d ig    W  t  s  g n  ü  V  h  s h a k it.
– Anpassung: Einst ll a , u  K  ativität  d   P äzisi n zu st ig  n.
P   pt Engin   ing: T  p  atu ...



C:\Users\amuel\AppData\Local\Temp\ipykernel_26236\2010197779.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents("Prädikatenlogik")


In [13]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()


chain = (
{
    "context": retriever,
    "question": RunnablePassthrough(),
}
    | LERNASSISTENT_PROMPT
    | llm
    | StrOutputParser()
)


In [14]:
from langchain_core.output_parsers import StrOutputParser

# USER_PROMPT="Generiere mir einen Post für die PubTour am 16. Oktober"

# result = chain.invoke("Generiere mir einen Post für die PubTour am 16. Oktober")
# print(result)

result = chain.invoke("Was ist Prädikatenlogik?")
print(result)

#result = chain.invoke(user_prompt)
#print(result)

Dazu habe ich im bereitgestellten Material nichts.  
Bitte laden Sie die Folie oder den Abschnitt hoch, in dem Prädikatenlogik behandelt wird (z. B. Titel / Seite / Kapitel).


## UI

In [15]:
import gradio as gr

def answer(question: str) -> str:
    # Kein LangSmith, einfach direkt die Chain ausführen
    try:
        response = chain.invoke(question)
        return response
    except Exception as e:
        return f"⚠️ Fehler bei der Verarbeitung: {e}"

# --- Gradio UI ---
demo = gr.Interface(
    fn=answer,
    inputs=gr.Textbox(label="Question", placeholder="Type your question here..."),
    outputs=gr.Textbox(label="Answer", lines=10),
    title="Lernassistent BAI",
    description="Stelle Fragen zum Lernstoff der BAI und erhalte präzise, prüfungsrelevante Antworten.",
)

# if __name__ == "__main__":
demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=56a0a349-7f2e-43e5-8d52-5ded467f6e9c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>